In [ ]:
!pip install adversarial-robustness-toolbox

In [ ]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Disable TensorFlow eager execution:
import tensorflow as tf
if tf.executing_eagerly():
    tf.compat.v1.disable_eager_execution()

# Load Keras dependencies:
from keras.applications.resnet import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.utils import load_img, img_to_array, array_to_img

# Load ART dependencies:
from art.estimators.classification import KerasClassifier
import art.attacks.evasion as art
from art.defences.preprocessor import SpatialSmoothing
from art.utils import to_categorical

from PIL import Image, ImageChops, ImageEnhance, ImageFilter
from array import *
import os

In [ ]:
def check(pred, real):
  label = decode_predictions(pred)
  if label[0][0][1] == real:
    check = 1
  else:
    check = 0
  return check

In [ ]:
model = ResNet50(weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
from art.preprocessing.preprocessing import Preprocessor

class ResNet50Preprocessor(Preprocessor):

    def __call__(self, x, y=None):
        return preprocess_input(x.copy()), y

    def estimate_gradient(self, x, gradient):
        return gradient[..., ::-1]

In [ ]:
preprocessor = ResNet50Preprocessor()
classifier = KerasClassifier(clip_values=(0, 255), model=model, preprocessing=preprocessor)

In [ ]:
directory_path = '/content/'
save_path = '/content/images'

In [ ]:
real = [[139, 'ruddy_turnstone'], [144, 'pelican'], [148, "killer_whale"], [149, "dugong"], [150, "sea_lion"], [152, "Japanese_spaniel"], [153, "Maltese_dog"], [154, "Pekinese"], [155, "Shih-Tzu"], [159, "Rhodesian_ridgeback"], [161, "basset"], [162, "beagle"], [164, "bluetick"], [166, "Walker_hound"], [168, "redbone"], [184, "Irish_terrier"], [185, "Norfolk_terrier"], [186, "Norwich_terrier"], [187, "Yorkshire_terrier"], [188, "wire-haired_fox_terrier"], [190, "Sealyham_terrier"], [191, "Airedale"], [193, "Australian_terrier"], [199, "Scotch_terrier"], [207, "golden_retriever"], [208, "Labrador_retriever"], [212, "English_setter"], [213, "Irish_setter"], [215, "Brittany_spaniel"], [222, "kuvasz"], [224, "groenendael"], [226, "briard"], [232, "Border_collie"], [234, "Rottweiler"], [240, "Appenzeller"], [241, "EntleBucher"], [243, "bull_mastiff"], [246, "Great_Dane"], [249, "malamute"], [250, "Siberian_husky"], [252, "affenpinscher"], [253, "basenji"], [254, "pug"], [255, "Leonberg"], [256, "Newfoundland"], [258, "Samoyed"], [259, "Pomeranian"], [261, "keeshond"], [262, "Brabancon_griffon"], [263, "Pembroke"], [219, "cocker_spaniel"], [266, "miniature_poodle"], [268, "Mexican_hairless"], [269, "timber_wolf"], [270, "white_wolf"], [272, "coyote"], [276, "hyena"], [277, "red_fox"], [279, "Arctic_fox"], [280, "grey_fox"], [281, "tabby"], [283, "Persian_cat"], [284, "Siamese_cat"], [285, "Egyptian_cat"], [287, "lynx"], [288, "leopard"], [289, "snow_leopard"], [290, "jaguar"], [291, "lion"], [292, "tiger"], [293, "cheetah"], [294, "brown_bear"], [295, "American_black_bear"], [296, "ice_bear"], [297, "sloth_bear"], [299, "meerkat"], [302, "ground_beetle"], [304, "leaf_beetle"], [305, "dung_beetle"], [306, "rhinoceros_beetle"], [307, "weevil"], [321, "admiral"], [323, "monarch"], [332, "Angora"], [333, "hamster"], [335, "fox_squirrel"], [336, "marmot"], [338, "guinea_pig"], [344, "hippopotamus"], [347, "bison"], [349, "bighorn"], [352, "impala"], [354, "Arabian_camel"], [358, "polecat"], [359, "black-footed_ferret"], [364, "three-toed_sloth"], [365, "orangutan"], [366, "gorilla"], [367, "chimpanzee"], [368, "gibbon"]]

In [ ]:
check_sum = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
RESNET_MEAN = np.array([103.939, 116.779, 123.68])

In [ ]:
eps = 1.
delt = 0.1
check_prob = 0.7

In [ ]:
def orthogonal_perturbation(delta, prev_sample, target_sample, mask):
	"""Generate orthogonal perturbation."""
	perturb = np.random.randn(1, 224, 224, 3)
	perturb /= np.linalg.norm(perturb, axis=(1, 2))
	perturb *= delta * np.mean(get_diff(target_sample, prev_sample))
	# Project perturbation onto sphere around target
	diff = (target_sample - prev_sample).astype(np.float32) # Orthorgonal vector to sphere surface
	diff /= get_diff(target_sample, prev_sample) # Orthogonal unit vector
	# We project onto the orthogonal then subtract from perturb
	# to get projection onto sphere surface
	perturb -= (np.vdot(perturb, diff) / np.linalg.norm(diff)**2) * diff
	# Check overflow and underflow
	overflow = (prev_sample + perturb) - 255 + RESNET_MEAN
	perturb -= overflow * (overflow > 0)
	underflow = -RESNET_MEAN
	perturb += underflow * (underflow > 0)
	if np.any(mask):
		perturb *= (1 - mask)
	return perturb

In [ ]:
def forward_perturbation(epsilon, prev_sample, target_sample, mask):
	"""Generate forward perturbation."""
	perturb = (target_sample - prev_sample).astype(np.float32)
	perturb *= epsilon
	#if np.any(mask):
		#perturb *= (1 - mask)
	return perturb

In [ ]:
def get_converted_prediction(sample, classifier):
	"""
	The original sample is dtype float32, but is converted
	to uint8 when exported as an image. The loss of precision
	often causes the label of the image to change, particularly
	because we are very close to the boundary of the two classes.
	This function checks for the label of the exported sample
	by simulating the export process.
	"""
	sample = (sample + RESNET_MEAN).astype(np.uint8).astype(np.float32) - RESNET_MEAN
	label = decode_predictions(classifier.predict(sample), top=1)[0][0][1]
	return label

In [ ]:
def preprocess(img):
	"""Load and preprocess image file."""
	x = image.img_to_array(img)
	x = np.expand_dims(x, axis=0)
	x = preprocess_input(x)
	return x

In [ ]:
def get_diff(sample_1, sample_2):
	"""Channel-wise norm of difference between samples."""
	return np.linalg.norm(sample_1 - sample_2, axis=(1, 2))

In [ ]:
def generate_mask(image_shape, probability):
  prob = 1 - probability
# Создаем маску, заполненную нулями
  mask = np.zeros(image_shape, dtype=np.float32)
# Выполняем случайный выбор для каждого пикселя с вероятностью probability
  for i in range(image_shape[1]):
    for j in range(image_shape[2]):
      if np.random.rand() < prob:
        mask[0, i, j, :] = 1 # Устанавливаем все каналы пикселя в 1
  return mask

In [ ]:
def restore_pixels(original_sample, perturbed_sample, mask):
  restored_sample = original_sample * mask + perturbed_sample * (1 - mask)
  return restored_sample

In [ ]:
def print_prediction(predictions):
  labels = decode_predictions(predictions)
  for i in range(5):
    kind = labels[0][i][1].replace("_", " ").title()
    percent = round(labels[0][i][2] * 100, 2)
    print(f"This is a {kind}. I am {percent} % sure.")

In [ ]:
def boundary_attack(classifier, target_sample, folder_to_save, prob, epsilon, delta):
	# Load model, images and other parameters
	# classifier = ResNet50(weights='imagenet')
	initial_sample = np.random.rand(1, 224, 224, 3)
	#target_sample = preprocess('/content/2.jpg')
	#folder = folder_to_save
	# save_image(np.copy(initial_sample), classifier, folder)
	attack_class = np.argmax(classifier.predict(target_sample))
	# print('Attack_class: ', attack_class)
	attack_prob = np.max(classifier.predict(target_sample))
	# print('Attack_prob: ', attack_prob)
	# target_class = np.argmax(classifier.predict(target_sample))
	mask = np.zeros((1, 224, 224, 3), dtype=np.float32)
	min_diff = 2000
	very_good_diff = 3000
	very_good_adversarial_sample = None
	good_adversarial_sample = None
	good_diff = 3000
	# count_diff = 0
	# goods = []

	adversarial_sample = initial_sample
	n_steps = 0
	n_calls = 0
	# epsilon = 1.
	# delta = 0.1

	# Move first step to the boundary
	while True:
		trial_sample = adversarial_sample + forward_perturbation(epsilon, adversarial_sample, target_sample, mask)
		prediction = classifier.predict(trial_sample)
		n_calls += 1
		if np.argmax(prediction) != attack_class:
			adversarial_sample = trial_sample
			break
		else:
			epsilon *= 0.9

	mask_ort = generate_mask((1, 224, 224, 3), prob)
	adversarial_sample = restore_pixels(target_sample, adversarial_sample, mask_ort)
	# Iteratively run attack
	while True:
		# print("Step #{}...".format(n_steps))
		# Orthogonal step
		# print("\tDelta step...")
		d_step = 0
		while True:
			d_step += 1
			# print("\t#{}".format(d_step))
			trial_samples = []
			#mask = generate_mask((1, 224, 224, 3), 0.85)
			for i in np.arange(10):
				trial_sample = adversarial_sample + orthogonal_perturbation(delta, adversarial_sample, target_sample, mask_ort)
				trial_samples.append(trial_sample)
			trial_samples_combined = np.concatenate(trial_samples, axis=0)
			predictions = classifier.predict(trial_samples_combined)
			n_calls += 10
			# print(np.max(predictions, axis=1))
			predictions = np.argmax(predictions, axis=1)
			predictions_prob = np.max(predictions)
			d_1 = np.mean((predictions != attack_class) & (abs(predictions - attack_class) < 30))
			#print(d_1)
			d_2 = np.mean((predictions == attack_class) & ((attack_prob - predictions_prob) > 5))
			d_score = d_1 + d_2
			if d_score > 0.0:
				if d_score < 0.3:
					delta *= 0.9
				elif d_score > 0.7:
					delta /= 0.9
				#adversarial_sample = np.array(trial_samples)[np.where(predictions != attack_class)[0][0]]
				adversarial_sample = np.array(trial_samples)[np.where(((predictions != attack_class) & (abs(predictions - attack_class) < 30))|((predictions == attack_class)&((attack_prob - predictions_prob) > 5)))[0][0]]
				break
			else:
				# print('Delta = ', delta)
				delta *= 0.9
			if d_step > 250:
				break
		# Forward step
		# print("\tEpsilon step...")
		e_step = 0
		while True:
			e_step += 1
			# print("\t#{}".format(e_step))
			mask = np.zeros((1, 224, 224, 3), dtype=np.float32)
			trial_sample = adversarial_sample + forward_perturbation(epsilon, adversarial_sample, target_sample, mask)
			prediction = classifier.predict(trial_sample)
			# print(np.argmax(prediction))
			n_calls += 1
			if (((np.argmax(prediction) != attack_class) & (abs(np.argmax(prediction) - attack_class) < 30))|((np.argmax(prediction) == attack_class)&((attack_prob - np.max(prediction)) > 5))):
			#if np.argmax(prediction) != attack_class:
				adversarial_sample = trial_sample
				epsilon /= 0.5
				break
			elif e_step > 250:
					break
			else:
				epsilon *= 0.5

		n_steps += 1
		#chkpts = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100, 150, 200, 500]
		#adversarial_sample = restore_pixels(target_sample, adversarial_sample, mask)
		#if (n_steps in chkpts) or (n_steps % 500 == 0):
			#print("{} steps".format(n_steps))
			#print("Original:")
			#print_prediction(classifier.predict(target_sample))
			#print("Adversarial:")
			#print_prediction(classifier.predict(adversarial_sample))
			#save_image(np.copy(adversarial_sample), classifier, folder)
			#diff = np.mean(get_diff(adversarial_sample, target_sample))
			#print('Difference: ', diff)
		diff = np.mean(get_diff(adversarial_sample, target_sample))
		#print('Diff: ', diff)
		#print('Good_diff: ', good_diff)
		if ((diff <= very_good_diff) & (np.argmax(prediction) != attack_class) & (diff >= min_diff)):
			very_good_diff = diff
			very_good_adversarial_sample = adversarial_sample
		if ((diff <= good_diff) & (diff >= min_diff)):
			good_diff = diff
			good_adversarial_sample = adversarial_sample
		if e_step > 250:
			# print("{} steps".format(n_steps))
			# print("Mean Squared Error: {}".format(diff))
			# save_image(np.copy(adversarial_sample), classifier, folder)
			break
		if n_steps > 100:
			#print("{} steps".format(n_steps))
			# save_image(np.copy(adversarial_sample), classifier, folder)
			break

		# print("Mean Squared Error: {}".format(diff))
		# print("Calls: {}".format(n_calls))
		# print("Attack Class: {}".format(attack_class))
		# print("Target Class: {}".format(target_class))
		# print("Adversarial Class: {}".format(np.argmax(prediction)))
		# print('Good: ', count_diff)

	if (very_good_adversarial_sample is not None):
		#print("Difference: {}".format(np.mean(get_diff(very_good_adversarial_sample, target_sample))))
		#print("Original:")
		#print_prediction(classifier.predict(target_sample))
		#print("Adversarial:")
		#print_prediction(classifier.predict(very_good_adversarial_sample))
		#label = get_converted_prediction(np.copy(very_good_adversarial_sample), classifier)
		#very_good_adversarial_sample_img = very_good_adversarial_sample[0]
		#very_good_adversarial_sample_img += RESNET_MEAN
		#very_good_adversarial_sample_img = very_good_adversarial_sample_img[..., ::-1].astype(np.uint8)
		#very_good_adversarial_sample_img = Image.fromarray(very_good_adversarial_sample_img)
		#very_good_adversarial_sample_img.save(os.path.join("images", folder, "{}_{}_{}.jpg".format('Very_good', label, prob)))
		return very_good_adversarial_sample
	elif (good_adversarial_sample is not None):
		#print("Difference: {}".format(np.mean(get_diff(good_adversarial_sample, target_sample))))
		#print("Original:")
		#print_prediction(classifier.predict(target_sample))
		#print("Adversarial:")
		#print_prediction(classifier.predict(good_adversarial_sample))
		#label = get_converted_prediction(np.copy(good_adversarial_sample), classifier)
		#good_adversarial_sample_img = good_adversarial_sample[0]
		#good_adversarial_sample_img += RESNET_MEAN
		#good_adversarial_sample_img = good_adversarial_sample_img[..., ::-1].astype(np.uint8)
		#good_adversarial_sample_img = Image.fromarray(good_adversarial_sample_img)
		#good_adversarial_sample_img.save(os.path.join("images", folder, "{}_{}_{}.jpg".format('Good', label, prob)))
		return good_adversarial_sample
	else:
		#print("Difference: {}".format(np.mean(get_diff(adversarial_sample, target_sample))))
		#print("Original:")
		#print_prediction(classifier.predict(target_sample))
		#print("Adversarial:")
		#print_prediction(classifier.predict(adversarial_sample))
		#label = get_converted_prediction(np.copy(adversarial_sample), classifier)
		#adversarial_sample_img = adversarial_sample[0]
		#adversarial_sample_img += RESNET_MEAN
		#adversarial_sample_img = adversarial_sample_img[..., ::-1].astype(np.uint8)
		#adversarial_sample_img = Image.fromarray(adversarial_sample_img)
		#adversarial_sample_img.save(os.path.join("images", folder, "{}_{}_{}.jpg".format('NotGood', label, prob)))
		return adversarial_sample

In [ ]:
img = load_img('/content/2.jpg', target_size=(224, 224))
x = np.expand_dims(img.copy(), axis=0)
x = preprocess_input(x)
x_art = np.expand_dims(img, axis=0)
adv = art.SimBA(classifier, max_iter = 600, epsilon = 100)
x_art_adv = adv.generate(x_art)
adv_img = (x_art_adv + 255) / 2
adv_img = adv_img.reshape(224, 224, 3)
adv_image = array_to_img(adv_img)
r, g, b = adv_image.split()
adv_image = Image.merge("RGB", (r, g, b))
target_boundary = preprocess(adv_image)
adv_Boundary = boundary_attack(model, target_boundary, save_path, check_prob, eps, delt)
adversarial_sample_img = adv_Boundary[0]
adversarial_sample_img += RESNET_MEAN
adversarial_sample_img = adversarial_sample_img[..., ::-1].astype(np.uint8)
advs_img = Image.fromarray(adversarial_sample_img)

In [ ]:
img = load_img('/content/1.jpg', target_size=(224, 224))
x = np.expand_dims(img.copy(), axis=0)
x = preprocess_input(x)
x_art = np.expand_dims(img, axis=0)
adv = art.SimBA(classifier, max_iter = 600, epsilon = 100)
x_art_adv = adv.generate(x_art)
adv_img = (x_art_adv + 255) / 2
adv_img = adv_img.reshape(224, 224, 3)
adv_image = array_to_img(adv_img)
r, g, b = adv_image.split()
adv_image = Image.merge("RGB", (r, g, b))
target_boundary = preprocess(adv_image)
adv_Boundary = boundary_attack(model, target_boundary, save_path, check_prob, eps, delt)
adversarial_sample_img = adv_Boundary[0]
adversarial_sample_img += RESNET_MEAN
adversarial_sample_img = adversarial_sample_img[..., ::-1].astype(np.uint8)
advs_img = Image.fromarray(adversarial_sample_img)

In [ ]:
for i in range(1, 101):
  file_name = f"{i}.jpg"
  file_path = os.path.join(directory_path, file_name)
  if os.path.exists(file_path):
    img = load_img(file_path, target_size=(224, 224))
    x_art = np.expand_dims(img, axis=0)
    adv = art.SimBA(classifier, max_iter = 600, epsilon = 100)
    x_art_adv = adv.generate(x_art)
    adv_img = (x_art_adv + 255) / 2
    adv_img = adv_img.reshape(224, 224, 3)
    adv_image = array_to_img(adv_img)
    r, g, b = adv_image.split()
    adv_image = Image.merge("RGB", (r, g, b))
    target_boundary = preprocess(adv_image)
    adv_Boundary = boundary_attack(model, target_boundary, save_path, check_prob, eps, delt)
    adversarial_sample_img = adv_Boundary[0]
    adversarial_sample_img += RESNET_MEAN
    adversarial_sample_img = adversarial_sample_img[..., ::-1].astype(np.uint8)
    advs_img = Image.fromarray(adversarial_sample_img)
    enhancer_bright = ImageEnhance.Brightness(advs_img)
    enhancer_color = ImageEnhance.Color(advs_img)
    im_rotate_1 = advs_img.rotate(15, expand=True)
    im_rotate_1.save("/content/mod_images/1_1.jpg")
    img_rotate_1 = load_img("/content/mod_images/1_1.jpg", target_size=(224, 224))
    img_rotate_1_art = np.expand_dims(img_rotate_1, axis=0)
    pred_1 = classifier.predict(img_rotate_1_art)
    check_sum[0] += check(pred_1, real[i-1][1])
    im_rotate_2 = advs_img.rotate(345, expand=True)
    im_rotate_2.save("/content/mod_images/1_2.jpg")
    img_rotate_2 = load_img("/content/mod_images/1_2.jpg", target_size=(224, 224))
    img_rotate_2_art = np.expand_dims(img_rotate_2, axis=0)
    pred_2 = classifier.predict(img_rotate_2_art)
    check_sum[1] += check(pred_2, real[i-1][1])
    light_img = enhancer_bright.enhance(1.5)
    light_img_art = np.expand_dims(light_img, axis=0)
    pred_3 = classifier.predict(light_img_art)
    check_sum[2] += check(pred_3, real[i-1][1])
    dark_img = enhancer_bright.enhance(0.5)
    dark_img_art = np.expand_dims(dark_img, axis=0)
    pred_4 = classifier.predict(dark_img_art)
    check_sum[3] += check(pred_4, real[i-1][1])
    small_img = advs_img.resize((100, 100))
    small_img.save("/content/mod_images/1_3.jpg")
    small_img = load_img("/content/mod_images/1_3.jpg", target_size=(224, 224))
    small_img_art = np.expand_dims(small_img, axis=0)
    pred_5 = classifier.predict(small_img_art)
    check_sum[4] += check(pred_5, real[i-1][1])
    big_img = advs_img.resize((1000, 1000))
    big_img.save("/content/mod_images/1_4.jpg")
    big_img = load_img("/content/mod_images/1_4.jpg", target_size=(224, 224))
    big_img_art = np.expand_dims(big_img, axis=0)
    pred_6 = classifier.predict(big_img_art)
    check_sum[5] += check(pred_6, real[i-1][1])
    bright_img = enhancer_color.enhance(1.5)
    bright_img_art = np.expand_dims(bright_img, axis=0)
    pred_7 = classifier.predict(bright_img_art)
    check_sum[6] += check(pred_7, real[i-1][1])
    chb_img = enhancer_color.enhance(0.0)
    chb_img_art = np.expand_dims(chb_img, axis=0)
    pred_8 = classifier.predict(chb_img_art)
    check_sum[7] += check(pred_8, real[i-1][1])
    clear_img = advs_img.filter(ImageFilter.SHARPEN)
    clear_img_art = np.expand_dims(clear_img, axis=0)
    pred_9 = classifier.predict(clear_img_art)
    check_sum[8] += check(pred_9, real[i-1][1])
    blurry_img = advs_img.filter(ImageFilter.GaussianBlur(radius=0.7))
    blurry_img_art = np.expand_dims(blurry_img, axis=0)
    pred_10 = classifier.predict(blurry_img_art)
    check_sum[9] += check(pred_10, real[i-1][1])
    print(check_sum)
print(check_sum)
SimBA_Boundary_result = 0
for i in range(0, 10):
  SimBA_Boundary_result += check_sum[i]
print("Среднее для атаки SimBA+Boundary: ", SimBA_Boundary_result / 1000)